## IMDB Scraping

I used basic html parser library of python to parse the html content of a film. To do that, I implemented htmlparser interface to recognize wanted attribute names of divisions(for title, storylines, and recommendations) and headers. I set flags in the handle start tag functions. According to flags I read the data in handle data functions. A special case occured for recommendations because recommended films are placed as attribute values not the data of tags. In the handle end tag I reset the flags.

In [2]:
import requests as req
# Gets the movie content of given film with id -> imdb_id
# I used basic html parser to parse the html of film
# When the code sees a div with wanted attirbutes it sets the read flags and if all the flags related to data are on 
# the data then will be read.
def get_movie_contents(imdb_id):
    # gets the html of site.
    r = req.get('https://www.imdb.com/title/'+imdb_id)
    from html.parser import HTMLParser
    # a parser class for parsing only title, body, and recomendatition parts of the film and their ids.
    class MyHTMLParser(HTMLParser):
        def __init__(self):
            HTMLParser.__init__(self)
            # attribute flag to read
            self.isTitleStoryLine = False
            self.isInlineCanWrap = False
            self.isTitleWraooer = False
            self.readTitleNow = False
            self.docs = {"title": "", "storyline": "", "recommendations": []}
        #setting flags true
        def handle_starttag(self, tag, attrs):
            if tag == 'div':
                for at in attrs:
                    if(at[0] == 'id' and at[1] == 'titleStoryLine'):
                        self.isTitleStoryLine = True
                    if(at[0] == 'class' and at[1] == 'inline canwrap'):
                        self.isInlineCanWrap = True
                    if(at[0] == 'class' and at[1] == 'title_wrapper'):
                        self.isTitleWraooer = True
                    if(at[0] == 'class' and at[1] == 'rec_item'):
                        for a in attrs:
                            if a[0] == 'data-tconst':
                                self.docs["recommendations"].append(a[1])
                        
            if self.isTitleWraooer and tag == 'h1':
                self.readTitleNow = True
           

        #handles the data if related flags are on
        def handle_data(self, data):
            if self.isInlineCanWrap and self.isTitleStoryLine:
                self.docs["storyline"] = self.docs["storyline"] + data
            if self.readTitleNow:
                self.docs["title"] = self.docs["title"] + data
                self.readTitleNow = False
        #resets the flags
        def handle_endtag(self, tag):
            if tag == 'div':
                self.isTitleWraooer = self.isInlineCanWrap = self.isTitleStoryLine = False
            if tag == 'h1':
                self.readTitleNow = False
    parser = MyHTMLParser()
    ##feeding parser with html content of the film
    parser.feed(r.text)
    return parser.docs

Gets all the movie ids and runs the parser. Creates movies dictionary which holds id, title, storyline, and recomendations. It can give connection timeot error due to connection errors. If you come across with such error please run again until it handles all the ids. 

In [3]:
import csv
# all movies dictionary. All the neccessary information will be added to this
movies = {}
with open('movie_ids.csv') as csv_file:
    csv_reader = csv.reader(csv_file)
    i = 1
    for row in csv_reader:
        #movie m. row[0] : id
        m = get_movie_contents(row[0])
        print(i, m['title'])
        i+=1
        movies[row[0]] = m


1 Desperado 
2 Kung Fu Panda 3 
3 Garfield 
4 Mortal Kombat: The Journey Begins 
5 Back to the Future Part III 
6 X-Men: Days of Future Past 
7 Vertigo 
8 Love, Rosie 
9 Ask the Dust 
10 Môsô dairinin             
11 The King's Speech 
12 Stuart Little 
13 Il castello dei morti vivi 
14 Within the Whirlwind 
15 Captain Kronos - Vampire Hunter 
16 The Flintstones in Viva Rock Vegas 
17 Sommarnattens leende 
18 Giulietta degli spiriti 
19 The Stranglers of Bombay 
20 Ansikte mot ansikte 
21 We Think the World of You 
22 Silver Linings Playbook 
23 Mister Cory 
24 Star Wars: Episode VII - The Force Awakens 
25 Meet Wally Sparks 
26 Why Worry? 
27 Kôkaku kidôtai: Stand Alone Complex             
28 Mean Girls 
29 Schatjes! 
30 Charlie Brown's All Stars! 
31 Batman: The Dark Knight Returns, Part 2 
32 Clash of the Titans 
33 Good Will Hunting 
34 Jingle All the Way 
35 From Beyond the Grave 
36 I, Monster 
37 Radio Days 
38 Frankenstein and the Monster from Hell 
39 Karins ansikte 
40 Toy S

354 Terminator 2: Judgment Day 
355 Pinocchio's Christmas 
356 Fury 
357 Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb 
358 The Night of the Following Day 
359 The Bachelor and the Bobby-Soxer 
360 Mediterraneo 
361 Final Destination 2 
362 Luxo Jr. 
363 Over the Hedge 
364 L'amore in città 
365 Bud Abbott Lou Costello Meet Frankenstein 
366 The Polar Express 
367 Scream 4 
368 Planes, Trains & Automobiles 
369 Lover Come Back 
370 Reservoir Dogs 
371 Interiors 
372 We Were Soldiers 
373 The Hunger Games: Mockingjay - Part 2 
374 Star Wars: Episode I - The Phantom Menace 
375 The Purple Rose of Cairo 
376 Drácula 
377 Love and Death 
378 Stolen 
379 Madagascar 
380 Life or Something Like It 
381 Det regnar på vår kärlek 
382 The Velocity of Gary 
383 La victoire en chantant 
384 Le testament d'Orphée, ou ne me demandez pas pourquoi! 
385 Crisis 
386 Ghost Rider 
387 Batman Begins 
388 Jurassic Park 
389 The Last Samurai 
390 Fifty Shades Freed 
391 The Grass Is G

699 Work 
700 Jetsons: The Movie 
701 13 Hours 
702 Melinda and Melinda 
703 L'aigle à deux têtes 
704 The Terminator 
705 The Ugly American 
706 Nomads 
707 Grado 3 
708 Jack the Giant Slayer 
709 V for Vendetta 
710 Kôkaku Kidôtai 
711 Twilight 
712 Dadnapped 
713 Asylum 
714 Home Alone 
715 Gone in Sixty Seconds 
716 The Cheetah Girls 
717 Manhattan Murder Mystery 
718 The Sword in the Stone 
719 Harry Potter and the Prisoner of Azkaban 
720 The Phantom of the Opera 
721 Black Hawk Down 
722 Monsters, Inc. 
723 The Great Mouse Detective 
724 Grandma's Boy 
725 X2 
726 Die seltsame Gräfin 
727 Free Willy 
728 Rear Window 
729 The Beach 
730 Rudolph the Red-Nosed Reindeer 
731 Metoroporisu 
732 Dracula Has Risen from the Grave 
733 Gycklarnas afton 
734 The Champion 
735 Tin Toy 
736 Son of Frankenstein 
737 La nuit du carrefour 
738 Star Wars: Episode VI - Return of the Jedi 
739 Her 
740 If I Stay 
741 Sherlock Holmes Faces Death 
742 Den Brother 
743 The Foot Shooting Party 
744 Th

1039 Markisinnan de Sade 
1040 Saw VI 
1041 Spider-Man: Homecoming 
1042 A King in New York 
1043 Friday the 13th Part III 
1044 The Lord of the Rings: The Return of the King 
1045 Home Alone 4: Taking Back the House 
1046 Hysteria 
1047 Pay Day 
1048 A Matter of Loaf and Death 
1049 Paths of Glory 
1050 Foxfire 
1051 The Cure 
1052 Space Jam 
1053 El Callejón de los Milagros 
1054 Lady and the Tramp 
1055 Attila Marcel 
1056 Urban Legend 
1057 Toni 
1058 Robots 
1059 Mojave Moon 
1060 A Clockwork Orange 
1061 Nostalgia 
1062 Kickin' It Old Skool 
1063 Big 
1064 The Avengers 
1065 Dumbo 
1066 Les beaux gosses 
1067 The Norliss Tapes 
1068 D & D & F: 'Dungeons & Dragons' & 'Futurama' 
1069 En lektion i kärlek 
1070 Fårö-dokument 1979 
1071 The Grinch 
1072 Captain America: Civil War 
1073 The Last Island 
1074 The Gorgon 
1075 Horton Hears a Who! 
1076 The Spirit of Christmas 
1077 Lolo 
1078 Alice in Wonderland 
1079 Fragmentos de Amor 
1080 Death Proof 
1081 Everest 
1082 Once Upon a 

### Preprocessing

Punctuation removal and case folding.

Basic preprocessing operations are handled here. Punctuation marks are replaced as whitespaces with the help of translate function and after that storylines splitted.

Please only run once because it changes storylines

In [ ]:
#punctuations to remove
punctuations = ['"', '!', '^', '%', '<', '+', '~', '*', ';', ':', '(', '?', '&', '}', ']', '|', ',', "'", ')', '-', '#', '`', '@', '/', '$', '_', '{', '.', '>', '[', '\\', '=']

#translations for punctuations.
trans = {}
for i in range(len(punctuations)):
    trans[ord(punctuations[i])] = ord(' ')
    
# case folding and punctuation removal of storylines.
for movie in movies:
    storyline = movies[movie]['storyline'].translate(trans).split()
    storyline = [x.lower() for x in storyline]
    movies[movie]['storyline'] = storyline

## Tf-idf model

Calculating tf and idfs of terms and encoding of storylines of the movies according to tf_idf scores. 

First I generated a vocabulary for the corpus of our films. To do that I used a set and updated it with each storyline of films. After generating the vocabulary of films I calculated the document frequencies for each term in the vocabulary. 

After finding the document frequencies. I calculated tf_idfs of each term in the storylines. To do that I traversed each term in a movie, found term frequencies and inverse document frequencies of that term. And if that tf_idf of that term exceeds a certain threshold I added that term to encoding of the film. To find that threshold I made two observations. Firstly, I checked some tf_idf scores of each term in a film and observe some values for unimportant words like 'but' , 'with' , 'as', 'who', 'and'. To ignore those terms I need to set threshold. Second observation was made done after completing evaluation part. I calculated average precision, recall and f1 scores for all movies and tried some thresholds and finalize my threshold with which gives the maximum average.

In [23]:
# vocabulary of document set
vocabulary = set()
# adds terms in storylines to vocabulary
for movie in movies:
    vocabulary.update(movies[movie]['storyline'])
#document frequency for each term in the vocabulary
dfs = {}
for term in vocabulary:
    dfs[term] = sum(1 for movie in movies if term in movies[movie]['storyline'])
    
import math
#encoding storylines of movies according to terms of these storylines. I used simmple term frequencies term count / terms and 
#logarithmically scaled inverse fraction of the documents that contain the word 
for movie in movies:
    l = movies[movie]['storyline']
    w = {}
    for term in l:
        #tfidf =         tf                  *               idf
        tf_idf = (l.count(term)*1.0 / len(l))*(math.log10(len(movies)*1.0/(dfs[term]+1)))
        # threshold
        if tf_idf > 0.01:
            w[term] = tf_idf
    if movie == 'tt0377092':
        #example tf_idf scores
        print(w)
    movies[movie]['w'] = w   

{'lindsay': 0.041296792513955335, 'lohan': 0.04409189186404551, 'stars': 0.029757130165760697, 'as': 0.013000073096277348, 'cady': 0.08818378372809102, 'heron': 0.04409189186404551, '16': 0.03545589116007288, 'year': 0.016853743552382033, 'old': 0.015368721790469836, 'homeschooled': 0.04409189186404551, 'girl': 0.01777425064726595, 'not': 0.013884114292744823, 'only': 0.013506864737859921, 'makes': 0.02087922523072875, 'mistake': 0.03453538406518897, 'falling': 0.03067763726064461, 'aaron': 0.04409189186404551, 'samuels': 0.04409189186404551, 'jonathan': 0.03545589116007288, 'bennett': 0.04409189186404551, 'ex': 0.026962030815670515, 'boyfriend': 0.027561849242486385, 'queenbee': 0.04409189186404551, 'regina': 0.08259358502791067, 'george': 0.027561849242486385, 'rachel': 0.03777538378988619, 'mcadams': 0.04409189186404551, 'also': 0.017850423391421014, 'unintentionally': 0.04409189186404551, 'joins': 0.028945185365008612, 'plastics': 0.041296792513955335, 'led': 0.02519773792483454, '

## Recommendation

Reccomends the ordered top 10 film according to cosine similirity. It returns top 10 because maximum K is 10.

To recommend films I simply traversed all the films other than the given film and calculated score for each film. After that sorted every film according to scores and returned the top 10 film(if films are less than 10 it returns all in ordered form). 

In [24]:
from collections import OrderedDict
def recommend(imdb_id):
    # movie content of given film  
    rec = movies[imdb_id]
    # tf_idfs of given film
    recWs = rec['w']
    # scores for each film
    scores = {}
    # length of vector for length normalization
    lengthOfW = sum(x**2 for x in recWs.values())**0.5
    for movie in movies:
        #if film in list is given film continue
        if movie == imdb_id:
            continue
        # compared film vector
        compareWs = movies[movie]['w']
        # length of other film
        lengthOfCompareW = sum(x**2 for x in compareWs.values())**0.5
        # intersection of given and compared film terms to make dot product
        termsToScore = set(recWs.keys()) & set(compareWs.keys())
        # score of the film
        score = 0
        for term in termsToScore:
            # dot product
            score += compareWs[term] * recWs[term]
        score = score * 1.0 / (lengthOfCompareW * lengthOfW)
        # assigning score to movie
        scores[movie] = score
    # sorts fims according to their scores.
    scores = OrderedDict(sorted(scores.items(), key=lambda x:x[1], reverse=True))
    if len(scores) < 10:
        return list(scores.keys())
    return list(scores.keys())[0:10]



In [25]:
# example run of recommend function
print(recommend('tt0356634'))

['tt0302596', 'tt0948470', 'tt0102558', 'tt0048280', 'tt0120363', 'tt0103786', 'tt4925292', 'tt0385586', 'tt1049413', 'tt0094737']


## Evaluation
I simply used general formulas for precision, recall and f1 score. I used set intersection to find number of movies which are mutual.

In [26]:
def evaluate_recommendations(rec_movie_ids, relevant_movie_ids, K):
    newRec_movie_ids = rec_movie_ids[0:K]
    p = len(set(newRec_movie_ids) & set(relevant_movie_ids))/K
    r = len(set(newRec_movie_ids) & set(relevant_movie_ids))/len(relevant_movie_ids)
    if r*p == 0:
        f1 = 0
    else:
        f1 = (2*r*p)/(r+p)
    return [p, r, f1]


In [27]:
#average scores for all movies for K = 10
ps = 0
rs = 0
f1s = 0
for movie in movies:
    #some movies have zero recommendations I checked that. 
    if len(movies[movie]['recommendations']) == 0:
        continue
    p, r, f1 = evaluate_recommendations(recommend(movie), movies[movie]['recommendations'],10)
    ps+=p
    rs+=r
    f1s+=f1
print(ps/len(movies),rs/len(movies),f1s/len(movies))

0.11676396997497868 0.10175055692153248 0.10794169697680668
